In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys, os
sys.path.insert(0, os.path.abspath("../../..")) 
from dust_extinction.parameter_averages import F19
from des_sn_hosts.simulations.spectral_utils import load_spectrum, convert_escma_fluxes_to_griz_mags,interpolate_SFH,interpolate_SFH_pegase
from des_sn_hosts.simulations.synspec import SynSpec, phi_t_pl
from des_sn_hosts.utils.utils import MyPool
import argparse
from astropy.cosmology import FlatLambdaCDM
import warnings
from astropy.utils.exceptions import AstropyWarning
from tables import NaturalNameWarning
warnings.filterwarnings('ignore', category=NaturalNameWarning)
np.seterr(all='ignore')
warnings.simplefilter('ignore', category=AstropyWarning)

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
cosmo = FlatLambdaCDM(70,0.3)
bc03_flux_conv_factor = 3.12e7

ModuleNotFoundError: No module named 'dust_extinction'

In [ ]:


# DTD parameters from W21
beta_x1hi = -1.68
norm_x1hi = 0.51E-13
beta_x1lo = -0.79
norm_x1lo = 1.19E-13
beta = -1.14
#beta=-1.5
dtd_norm = 2.08E-13


In [ ]:

def parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('-i','--input',help='Filename for input SFHs', default= '/Users/ishfahani/master_thesis/galaxy_sfh/SFHs_alt_ver2_0.5_quenched.h5')
    parser.add_argument('-z','--z',help='Redshift',default=0.5,type=str)
    parser.add_argument('-zl','--zlo',help='Redshift lower end',default=0.15,type=float)
    parser.add_argument('-zh','--zhi',help='Redshift upper end',default=1.25,type=float)
    parser.add_argument('-zs','--zstep',help='Redshift step',default=0.15,type=float)
    parser.add_argument('-al','--av_lo',help='Lowest Av',default=0,type=float)
    parser.add_argument('-ah','--av_hi',help='Highest Av',default=1,type=float)
    parser.add_argument('-na','--n_av',help='Av step',default=20,type=int)
    parser.add_argument('-at','--av_step_type',help='Av step type (lin or log)',default='lin')
    parser.add_argument('-u','--logU',help='Ionisation parameter',default=-2,type=float)
    parser.add_argument('-tr','--time_res',help='SFH time resolution',default=5,type=int)
    parser.add_argument('-t','--templates',help='Template library to use [BC03, PEGASE]',default='BC03',type=str)
    parser.add_argument('-tf','--templates_fn',help='Filename of templates',type=str,default='None')
    parser.add_argument('-ne','--neb',action='store_true')
    parser.add_argument('-b','--beta',help='Absolute value of the slope of the DTD',default=1.14,type=float)
    args, unknown = parser.parse_known_args()  
    return args


In [ ]:
def sed_worker(worker_args):
    sfh_df,args,av_arr,z,tf,s,bc03_logt_float_array,counter= [worker_args[i] for i in range(8)]
    print('Starting %i'%counter)
    try:
        results = []
        i = np.random.randint(0,len(sfh_df.index.unique()))
        sfh_iter_df = sfh_df.loc[i]

        mtot=sfh_iter_df['m_tot'].iloc[-1]
        age = sfh_iter_df['age'].iloc[-1]
        #print('Mass: ',np.log10(mtot),'age: ',age)
        ssfr = np.sum(sfh_iter_df['m_formed'].iloc[-500:])/((250*1E+6)*mtot)
        sfr = ssfr*mtot
        sfh_iter_df['stellar_age'] = sfh_iter_df.age.values[::-1]
        ages = sfh_iter_df['stellar_age']/1000
        dtd_x1hi = phi_t_pl(ages,0.04,beta_x1hi,norm_x1hi)
        pred_rate_x1hi =np.sum(sfh_iter_df['m_formed']*dtd_x1hi)
        dtd_x1lo = phi_t_pl(ages,0.04,beta_x1lo,norm_x1lo)
        pred_rate_x1lo =np.sum(sfh_iter_df['m_formed']*dtd_x1lo)
        dtd_total =phi_t_pl(ages,0.04,-1*args.beta,dtd_norm)
        SN_age_dist = sfh_iter_df['m_formed']*dtd_total
        pred_rate_total = np.sum(SN_age_dist)

        mwsa = np.average(sfh_iter_df['stellar_age'],weights=sfh_iter_df['m_formed']/mtot)
        if np.log10(mtot)<=9.5:
            mu_Rv = 2.61
        elif 9.5 <np.log10(mtot)<=10.5:
            mu_Rv = 2.99
            #avs_SBL =np.clip(np.random.normal(av_means_mhi(np.log10(mtot)),av_sigma(np.log10(mtot)),size=20),a_min=0,a_max=None)
        else:
            mu_Rv = 3.47
            #avs_SBL = np.clip(np.random.normal(av_means_mlo,av_sigma(np.log10(mtot)),size=20),a_min=0,a_max=None)
        if args.templates == 'BC03':
            sfh_coeffs_PW21 = interpolate_SFH(sfh_iter_df,mtot,bc03_logt_float_array)
            template=None
        elif args.templates =='PEGASE':
            if args.templates_fn =='None':
                templates = pd.read_hdf('/media/data3/wiseman/des/AURA/PEGASE/templates.h5',key='main')
            else:
                templates = pd.read_hdf(args.templates_fn,key='main')
            sfh_coeffs_PW21 = interpolate_SFH_pegase(sfh_iter_df,templates['time'],mtot,templates['m_star'])
        arr = np.zeros((len(ages),2))
        arr[:,0] = ages
        arr[:,1] = SN_age_dist
        np.savetxt('/Users/ishfahani/master_thesis/hostlib/all_model_params_quench_%s_z_%.5f_rv_rand_full_age_dists_neb_U%.2f_res_%i_beta_%.2f_%.1f.dat'%(args.templates,z,args.logU,args.time_res,args.beta,tf),arr)
        for Av in av_arr:
            Rv = np.min([np.max([2.0,np.random.normal(mu_Rv,0.5)]),6.0])
            delta='None'
            #if args.templates =='PEGASE':
            #    sfh_coeffs_PW21 = None
            #    template = pd.read_hdf('/media/data3/wiseman/des/AURA/PEGASE/templates_analytic_orig_%i.h5' % tf,
            #                           key='main')
            galid_string = 'all_model_params_quench_%s_z_%.5f_rv_rand_full_age_dists_neb_U%.2f_res_%i_beta_%.2f_%.1f_%.3f_%.3f'%(args.templates,z,args.logU,args.time_res,args.beta,tf,Av,Rv)


            U_R,fluxes,colours= s.calculate_model_fluxes_pw(z,sfh_coeffs_PW21,dust={'Av':Av,'Rv':Rv,'delta':'none','law':'CCM89'},
                                                    neb=args.neb,logU=args.logU,mtot=mtot,age=age,specsavename=galid_string)
            obs_flux  = list(fluxes.values())#+cosmo.distmod(z).value
            U,B,V,R,I,sdssu,sdssg,sdssr,sdssi,sdssz = (colours[i] for i in colours.keys())

            results.append([z,mtot,ssfr,mwsa,Av,Rv,delta,U_R[0],pred_rate_x1hi,pred_rate_x1lo,pred_rate_total,tf,
                    obs_flux[0],obs_flux[1],obs_flux[2],obs_flux[3],U,B,V,R,I,sdssu,sdssg,sdssr,sdssi,sdssz,galid_string])

        df = pd.DataFrame(results,columns=['z','mass','ssfr','mean_age','Av','Rv','delta','U_R','pred_rate_x1_hi',
                                                'pred_rate_x1_lo','pred_rate_total','t_f',
                                                'm_g','m_r','m_i','m_z','U','B','V','R','I','sdssu','sdssg','sdssr','sdssi','sdssz','galid_spec'])
        #df['g_r'] = df['m_g'] - df['m_r']

    except:
        return
    print('Saving %i'%counter)
    df.to_hdf('/Users/ishfahani/master_thesis/hostlib/all_model_params_quench_%s_z%.5f_%.5f_av%.2f_%.2f_rv_rand_full_age_dists_neb_U%.2f_res_%i_beta_%.2f_%.5f_%i_sdss_u_r.h5'%(args.templates,args.zlo,args.zhi,av_arr[0],av_arr[-1],args.logU,args.time_res,args.beta,z,tf),
        key='main')
    print('Returning %i'%counter)
    return

In [ ]:
def run(args):
    # DES filter objects

    filt_dir = '/Users/ishfahani/master_thesis/'
    filt_obj_list = [
        load_spectrum(filt_dir+'decam_g.dat'),
        load_spectrum(filt_dir+'decam_r.dat'),
        load_spectrum(filt_dir+'decam_i.dat'),
        load_spectrum(filt_dir+'decam_z.dat'),
    ]

    nfilt = len(filt_obj_list)

    aura_dir = '/Users/ishfahani/master_thesis/'
    #------------------------------------------------------------------------
    # BC03 SSPs as mc_spec Spectrum objects
    f1 = open(aura_dir+'/bc03_logt_list.dat')
    if args.templates =='BC03':
        bc03_logt_list = [x.strip() for x in f1.readlines()]
        f1.close()
        bc03_logt_array = np.array(bc03_logt_list)
        ntemp = len(bc03_logt_array)
        bc03_logt_float_array =np.array([float(x) for x in (bc03_logt_array)])
        bc03_dir = '/Users/ishfahani/master_thesis/bc03_ssp_templates/'
        template_obj_list = []
        nLy_list = []
        for i in range(ntemp):
            bc03_fn = '%sbc03_chabrier_z02_%s.spec' % (bc03_dir, bc03_logt_list[i])
            new_template_spec =  load_spectrum(bc03_fn)
            template_obj_list.append(new_template_spec)

        s = SynSpec(template_obj_list = template_obj_list,neb=args.neb)
        neb=args.neb
    elif args.templates=='PEGASE':
        s = SynSpec(library='PEGASE',template_dir = '/Users/ishfahani/master_thesis/bc03_ssp_templates/AURA/PEGASE/templates/',neb=args.neb)

        neb=args.neb
    store = pd.HDFStore(args.input,'r')
    ordered_keys = np.sort([int(x.strip('/').replace('m', '')) for x in store.keys() if x.strip('/').replace('m', '').isdigit()])

    #z_array = [float(z) for z in args.z.split(',')]
    z_array = np.arange(args.zlo,args.zhi,args.zstep)
    if args.av_step_type == 'lin':
        av_arr = np.linspace(args.av_lo,args.av_hi,args.n_av)
    elif args.av_step_type == 'log':
        av_arr = np.logspace(args.av_lo,args.av_hi,args.n_av)

    for z in tqdm(z_array):
        print('Making hostlib for z=%.2f'%z)
        worker_args = []
        for counter,tf in enumerate(ordered_keys[::-1][np.arange(0,len(ordered_keys),args.time_res)]):   # Iterate through the SFHs for galaxies of different final masses
            sfh_df = store['m%3.0f' % tf]
            sfh_df = sfh_df[sfh_df['z']>z]

            if len(sfh_df)>0:
                worker_args.append([sfh_df,args,av_arr,z,tf,s,bc03_logt_float_array,counter])
        pool_size = 8
        pool = MyPool(processes=pool_size)
        print('Sending %i jobs'%len(worker_args))
        results =pool.map_async(sed_worker,worker_args)

        pool.close()
        pool.join()

    print("Done!")



In [17]:
if __name__=="__main__":
    args = parser()
    run(args)

In [5]:
import pandas as pd
file_assembly= '/Users/ishfahani/master_thesis/galaxy_sfh/SFHs_alt_ver2_0.5_quenched.h5'
with pd.HDFStore(file_assembly, "r") as store:
    print("Available keys:", store.keys())


# Read the dataset
df = pd.read_hdf(file_assembly, key="/m9500")
print(df)

Available keys: ['/m1000', '/m10000', '/m10250', '/m10500', '/m10750', '/m11000', '/m11250', '/m11500', '/m11750', '/m12000', '/m12250', '/m12500', '/m12750', '/m1500', '/m2000', '/m2500', '/m3000', '/m3500', '/m4000', '/m4500', '/m5000', '/m5500', '/m6000', '/m6500', '/m7000', '/m7500', '/m8000', '/m8500', '/m9000', '/m9500']
         t         z     age       m_formed  final_age_weights         m_tot
0   9500.0  1.524750     0.0   10596.380787        5504.440416  1.010596e+06
0   9499.5  1.524543     0.5   10672.194632        5543.848831  1.020765e+06
0   9499.0  1.524337     1.0   10744.569924        5581.471394  1.031104e+06
0   9498.5  1.524130     1.5   10817.940493        5619.611326  1.041393e+06
0   9498.0  1.523924     2.0   10890.677027        5657.422251  1.051654e+06
..     ...       ...     ...            ...                ...           ...
99     2.0  0.000142  9498.0  312634.150172      282293.130714  4.692576e+09
99     1.5  0.000106  9498.5  312618.291988      285845